In [ ]:
import cv2
import numpy as np
from sklearn.metrics import pairwise
from google.colab.patches import cv2_imshow

In [ ]:

#Initialising global variable 
background =None
accumulated_weight = 0.5
#Selecting the Region of interest initially
roi_top = 50
roi_bottom = 500
roi_right = 500
roi_left = 800


In [ ]:
#For a stable background or to distinguish between background and foreground, we try to take the average of frames. 

def accu_avg(frame, accumulated_weight):
  global background

  #Checking just in case the background is not assigned yet.
  if background is None:
    background = frame.copy().astype("float")
    return None
  
  cv2.accumulateWeighted(frame, background, accumulated_weight)

In [ ]:
#This function is used to detect the hand contour from the frame or image provided.

def segment(frame,threshold=25):
  global background

  #Absolute difference is helpful inremoving the stationary background in the frame so that thresholding can give us a better look at the region of interest.
  diff = cv2.absdiff(background.astype("uint8"), frame)

  _ , thresh = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
  from google.colab.patches import cv2_imshow
  #We look for contours in the thresholded frame.
  image, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  
  if len(contours) == None:
    return None  #If there are no contours, then we exit the function with no fingers to count
  else:
    # If we detect the contours, then we would be looking for the biggeest one that will be signifying the hand
    hand_segment = max(contours,key = cv2.contourArea)

  return thresh , hand_segment




In [ ]:
# Function to count the fingers

def finger_count(thresh,hand_segment):

  # Draw the convex hull to detect the extreme points of the hand segment.
  hull = cv2.convexHull(hand_segment)

  # Extracting the extreme points of convex hull
  top    = tuple(hull[hull[:, :, 1].argmin()][0])
  bottom = tuple(hull[hull[:, :, 1].argmax()][0])
  left   = tuple(hull[hull[:, :, 0].argmin()][0])
  right  = tuple(hull[hull[:, :, 0].argmax()][0])

  # Approximating the co-orsdinates of the center of hand segment
  cX = (left[0] + right[0]) // 2
  cY = (top[1] + bottom[1]) // 2

  # Calculates the distance between center and extreme points of the hull.
  distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]

  # To further refine the region of interest, we take the max distance from the centre as the radius and make a circle of that around the center.
  max_distance = distance.max()
  radius = int(0.8 * max_distance)
  circumference = (2 * np.pi * radius)
  circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
  cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
  
  # The function then applies a bitwise AND operation to obtain only the pixels of the thresholded image that are inside the circular ROI.
  circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

  # It finds the countours in this ROI. 
  image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  count = 0

  # This loop checks if contours are not in the wrist region or not, by defining the region of wrist and checking if the countour lies in the region.
  for cnt in contours:
    (x, y, w, h) = cv2.boundingRect(cnt)
    out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
   
    #limit_points = ((circumference * 0.25) > cnt.shape[0])

    if  out_of_wrist :
       count += 1  # Finally if it is not in the wrist region, then it is a finger.

  return count



In [ ]:

# Initialize camera object
cam = cv2.VideoCapture("/VID_20230504_190552.mp4")

# Counter variable for number of frames processed
num_frames = 0

# Loop infinitely
while True:

    # Capture frame from camera
    ret, frame = cam.read()
    print("hello")
    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Create a copy of the frame for drawing purposes
    frame_copy = frame.copy()

    # Set region of interest (ROI) for detecting hand
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    # Convert ROI to grayscale and apply Gaussian blur to reduce noise
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # If the number of frames processed is less than 60, calculate the accumulated weighted average for background removal
    if num_frames < 60:
        accu_avg(gray, accumulated_weight)
        # Display "WAIT! GETTING BACKGROUND AVG." message during this time
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2_imshow(frame_copy)
            
    # If the number of frames processed is greater than or equal to 60, segment the hand and count the fingers
    else:
        # Segment the hand from the background
        hand = segment(gray)

        # If the hand is detected in the ROI, draw a contour around it and count the fingers
        if hand is not None:
            thresholded, hand_segment = hand
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)
            fingers = finger_count(thresholded, hand_segment)
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2_imshow(thresholded)

    # Draw the ROI on the frame
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    # Increment the number of frames processed
    num_frames += 1

    # Display the resulting frame
    cv2_imshow(frame_copy)

    # Wait for a keypress, and check if the Esc key was pressed to break the loop
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

# Release the camera object and destroy all windows
cam.release()
cv2.destroyAllWindows() 
